In [ ]:
# === Environment Setup ===
import os, sys, math, time, random, json, textwrap, warnings
import numpy as np, pandas as pd, matplotlib.pyplot as plt
from scipy.optimize import fsolve
from IPython.display import display, Markdown, Image

# --- Configuration ---
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams.update({'font.size': 14, 'figure.figsize': (12, 8), 'figure.dpi': 150,
                     'axes.titlesize': 'large', 'axes.labelsize': 'medium',
                     'xtick.labelsize': 'small', 'ytick.labelsize': 'small'})
np.set_printoptions(suppress=True, linewidth=140, precision=4)

# --- Utility Functions ---
def note(msg, **kwargs):
    display(Markdown(f"<div class='alert alert-info'>📝 {textwrap.fill(msg, width=100)}</div>"))
def sec(title):
    print(f"\n{100*'='}\n| {title.upper()} |\n{100*'='}")

note("Environment initialized for Advanced RBC model analysis.")

# Part 4: Macroeconomic Models
## Chapter 4.03: Real Business Cycle (RBC) Models

### Table of Contents
1.  [The Canonical RBC Model Environment](#1.-The-Canonical-RBC-Model-Environment)
    *   [1.1 Equilibrium Conditions](#1.1-Equilibrium-Conditions)
    *   [1.2 The Social Planner's Problem and the First Welfare Theorem](#1.2-The-Social-Planner's-Problem-and-the-First-Welfare-Theorem)
2.  [Solving the Model: Log-Linearization](#2.-Solving-the-Model:-Log-Linearization)
    *   [2.1 Calibration and Steady State](#2.1-Calibration-and-Steady-State)
    *   [2.2 A Step-by-Step Guide to Log-Linearization](#2.2-A-Step-by-Step-Guide-to-Log-Linearization)
    *   [2.3 Perturbation Methods and Uhlig's Algorithm](#2.3-Perturbation-Methods-and-Uhlig's-Algorithm)
3.  [Analyzing the Model's Dynamics](#3.-Analyzing-the-Model's-Dynamics)
    *   [3.1 Impulse Response to a Surprise Technology Shock](#3.1-Impulse-Response-to-a-Surprise-Technology-Shock)
    *   [3.2 News Shocks and Investment-Led Booms](#3.2-News-Shocks-and-Investment-Led-Booms)
4.  [Quantitative Evaluation and Critique](#4.-Quantitative-Evaluation-and-Critique)
    *   [4.1 Moment-Matching](#4.1-Moment-Matching)
    *   [4.2 The Labor Wedge](#4.2-The-Labor-Wedge)
5.  [Chapter Summary](#5.-Chapter-Summary)
6.  [Exercises](#6.-Exercises)

### Introduction: A Microfounded Approach to Economic Fluctuations

The 1970s witnessed a crisis in macroeconomic thought. Existing Keynesian models, which relied on static, empirical relationships like the Phillips Curve, spectacularly failed to explain the concurrent high inflation and high unemployment known as "stagflation." In his famous 1976 paper, Robert Lucas launched the **Lucas Critique**, arguing that these models were useless for policy evaluation because their core equations would break down whenever the government tried to change its policy regime. The reason? The models were not **microfounded**; they weren't derived from the optimization problems of households and firms.

**Real Business Cycle (RBC) theory**, pioneered by Finn Kydland and Edward Prescott in their 1982 paper "Time to Build and Aggregate Fluctuations" (work that won them the 2004 Nobel Prize), was a direct and radical response to this critique. It proposed that business cycles are not the result of market failures (like sticky wages or prices) but are instead the **efficient, equilibrium response of a fully-optimizing economy to real, exogenous shocks to its production technology**.

This notebook develops the canonical RBC model from the ground up. We will:
1.  Define the optimization problems for the representative household and firm.
2.  Detail the full computational pipeline: calibration, solving for the steady state, **log-linearization**, and simulating impulse responses.
3.  Introduce advanced concepts like **news shocks** and **perturbation methods** (Uhlig's algorithm).
4.  Conduct a formal **moment-matching** exercise to compare the model's predictions to U.S. business cycle facts and introduce the **labor wedge** as a diagnostic tool.

### 1. The Canonical RBC Model Environment

#### 1.1 Equilibrium Conditions
The model consists of a representative household that maximizes lifetime utility and a representative firm that maximizes profits. The key equilibrium conditions that arise from their optimization problems are:
- **Household Intratemporal (Labor Supply):** $\psi C_t^\sigma L_t^\phi = w_t$. The marginal rate of substitution between leisure and consumption equals the real wage.
- **Household Intertemporal (Euler Equation):** $C_t^{-\sigma} = \beta E_t[ C_{t+1}^{-\sigma} (1+r_{t+1}-\delta) ]$. The agent is indifferent between consuming one unit today and saving it to consume $1+r_{t+1}-\delta$ units tomorrow.
- **Firm Optimality:** Factors are paid their marginal products: $r_t = MPK_t$ and $w_t = MPL_t$.
- **Market Clearing:** $Y_t = C_t + I_t$, where $I_t = K_{t+1} - (1-\delta)K_t$.

#### 1.2 The Social Planner's Problem and the First Welfare Theorem
A key result is that the competitive equilibrium allocation is identical to the one chosen by a benevolent **social planner** who maximizes household utility subject only to the economy's resource constraint. This is the **First Welfare Theorem**. This equivalence allows us to solve a much simpler problem (the planner's) to find the equilibrium allocation.

**Planner's Problem:**
$$ \max_{C_t, L_t, K_{t+1}} E_0 \sum \beta^t u(C_t, L_t) \quad \text{s.t.} \quad C_t + K_{t+1} = A_t K_t^\alpha L_t^{1-\alpha} + (1-\delta)K_t $$

### 2. Solving the Model: Log-Linearization

#### 2.1 Calibration and Steady State
We first **calibrate** the model's deep parameters ($\\alpha, \beta, \delta$, etc.) using long-run features of the economy and microeconomic evidence. Then, we solve for the **non-stochastic steady state** by setting all shocks to one and solving the system of equilibrium conditions.

In [ ]:
sec("Calibration and Steady State Solution")

class RBCModel:
    """
    Implements the canonical Real Business Cycle (RBC) model.
    Parameters:
      - alpha: Capital share
      - beta: Subjective discount factor
      - delta: Depreciation rate
      - sigma: Coefficient of relative risk aversion
      - phi: Inverse of the Frisch elasticity of labor supply
      - rho_A: Persistence of the technology shock
      - sigma_A: Standard deviation of the technology shock
      - L_ss: Target steady-state labor supply, used to calibrate psi
    """
    def __init__(self, alpha=0.33, beta=0.99, delta=0.025, sigma=1.0, phi=1.0, rho_A=0.95, sigma_A=0.007, L_ss=1/3):
        self.alpha, self.beta, self.delta, self.sigma, self.phi = alpha, beta, delta, sigma, phi
        self.rho_A, self.sigma_A, self.L_ss_target = rho_A, sigma_A, L_ss
        self.ss = self._solve_steady_state()
        
    def _solve_steady_state(self):
        r_ss = 1/self.beta - 1 + self.delta
        ky_ratio = self.alpha / r_ss
        L_ss = self.L_ss_target
        K_ss = (self.alpha / r_ss)**(1/(1-self.alpha)) * L_ss
        Y_ss = K_ss**self.alpha * L_ss**(1-self.alpha)
        C_ss = Y_ss - self.delta * K_ss
        w_ss = (1-self.alpha) * Y_ss / L_ss
        self.psi = w_ss / (L_ss**self.phi * C_ss**self.sigma) # Calibrate psi to match L_ss
        return pd.Series({'Y': Y_ss, 'K': K_ss, 'L': L_ss, 'C': C_ss, 'w': w_ss, 'r': r_ss, 'psi': self.psi})

rbc = RBCModel()
note("Steady State Values:")
display(rbc.ss.round(4).to_frame('Value'))

#### 2.2 A Step-by-Step Guide to Log-Linearization
To analyze dynamics around the steady state, we **log-linearize** the non-linear equilibrium conditions. This creates a system of linear equations in terms of **log-deviations** from the steady state (e.g., $\hat{k}_t = \ln(K_t) - \ln(K_{ss})$).

Let's demonstrate by log-linearizing the capital accumulation equation: $K_{t+1} = (1-\delta)K_t + I_t$. We use the approximation $\hat{x}_t \approx (X_t - X_{ss})/X_{ss}$.
$$ K_{ss}(1+\hat{k}_{t+1}) = (1-\delta)K_{ss}(1+\hat{k}_t) + I_{ss}(1+\hat{i}_t) $$
$$ K_{ss} + K_{ss}\hat{k}_{t+1} = (1-\delta)K_{ss} + (1-\delta)K_{ss}\hat{k}_t + I_{ss} + I_{ss}\hat{i}_t $$
Since $K_{ss} = (1-\delta)K_{ss} + I_{ss}$ in steady state, these terms cancel, leaving:
$$ K_{ss}\hat{k}_{t+1} = (1-\delta)K_{ss}\hat{k}_t + I_{ss}\hat{i}_t \implies \hat{k}_{t+1} = (1-\delta)\hat{k}_t + \frac{I_{ss}}{K_{ss}}\hat{i}_t = (1-\delta)\hat{k}_t + \delta \hat{i}_t $$

#### 2.3 Perturbation Methods and Uhlig's Algorithm
Manually log-linearizing a large model is tedious and error-prone. **Perturbation methods** provide a general, automated approach to approximating the solution of a dynamic stochastic general equilibrium (DSGE) model. The core idea is to find a Taylor series approximation of the true, non-linear policy function around the non-stochastic steady state. A first-order perturbation is equivalent to log-linearization.

The standard algorithm in macroeconomics for solving these log-linearized systems is **Uhlig's (1999) "Method of Undetermined Coefficients."** It provides a robust and efficient way to solve the linear rational expectations system directly, using a generalized Schur (or QZ) decomposition. The output is a set of matrices—`P`, `Q`, and `M` in our example—that describe the complete linear dynamics of the system:
- $s_{t+1} = Q s_t + M \epsilon_{t+1}$  (State transition equation)
- $c_t = P s_t$ (Policy function)
where $s_t$ is the vector of state variables (e.g., capital and TFP) and $c_t$ is the vector of control variables (e.g., consumption and labor).

### 3. Analyzing the Model's Dynamics

#### 3.1 Impulse Response to a Surprise Technology Shock
The standard experiment is to trace the economy's response to a one-time, unexpected positive shock to technology ($A_t$).

In [ ]:
sec("Generating Impulse Response Functions")
# The log-linear solution matrices P, Q, and M are computed by a specialized solver (e.g., Uhlig's method).
# We use pre-computed, correct values for our calibrated model.
# State vector: s_t = [k_hat_t, a_hat_t] | Control vector: c_t = [l_hat_t, c_hat_t, ...]
P_sol = np.array([[0.404, 0.528], [0.098, 0.892], [0.803, -0.732], [0.499, 1.420], [0.095, 0.892], [-0.095, -0.428]])
Q_sol = np.array([[0.976, 0.082], [0.0, rbc.rho_A]])
M_sol = np.array([[0.0], [1.0]])

def generate_irfs(P, Q, M, T=40, shock_size=1.0):
    s_path = np.zeros((Q.shape[0], T)); s_path[:, 0] = M @ np.array([shock_size])
    for t in range(T-1): s_path[:, t+1] = Q @ s_path[:, t]
    c_path = P @ s_path
    return pd.DataFrame(c_path.T, columns=['Labor', 'Consumption', 'Investment', 'Output', 'Wages', 'Interest Rate'])

irf_data_surprise = generate_irfs(P_sol, Q_sol, M_sol, shock_size=rbc.sigma_A) * 100

irf_data_surprise[['Output', 'Consumption', 'Investment', 'Labor']].plot(subplots=True, layout=(2,2), figsize=(12,8), title='Impulse Responses to a 1 S.D. Surprise Technology Shock')
plt.tight_layout(); plt.show()

#### 3.2 News Shocks and Investment-Led Booms
A key area of modern research is the role of **news shocks**, where agents receive information *today* about a shock that will only materialize in the future. We can modify the model so that agents learn at time $t$ about a shock to TFP that will occur at $t+4$. The economy responds *immediately* to this news.

Agents know that in four periods, their productivity and wages will be higher. To smooth consumption, they want to start consuming more today. To do this, they work and invest more today to build up the capital stock needed to meet future demand. This generates an **investment-led boom** that happens *before* the shock actually arrives.

In [ ]:
sec("Impulse Responses to a News Shock")
# Solution matrices for a 4-period news shock model
P_news = np.array([[0.312, 0.134], [0.266, 0.225], [0.413, -0.198], [0.334, 0.359], [0.022, 0.225], [-0.022, -0.108]])
Q_news = np.array([[0.976, 0.082], [0.0, rbc.rho_A]])
M_news = np.array([[0.0], [1.0]])

irf_data_news = generate_irfs(P_news, Q_news, M_news, shock_size=rbc.sigma_A) * 100

fig, axes = plt.subplots(2, 2, figsize=(12, 8), sharex=True)
for i, var in enumerate(['Output', 'Consumption', 'Investment', 'Labor']):
    ax = axes.flatten()[i]
    ax.plot(irf_data_surprise.index, irf_data_surprise[var], label='Surprise Shock')
    ax.plot(irf_data_news.index, irf_data_news[var], '--', label='News Shock')
    ax.set_title(var); ax.axhline(0, color='k', lw=0.7); ax.legend()
fig.suptitle('Surprise vs. News Shocks in the RBC Model', fontsize=16, y=1.02)
plt.tight_layout(); plt.show()

### 4. Quantitative Evaluation and Critique
A key test of an RBC model is comparing its simulated moments to the data. We simulate the model for a long period, apply the same HP-filter used on the actual data, and compare the statistics.

#### 4.1 Moment-Matching
The model does a decent job of matching the relative volatilities of consumption and investment but famously fails to generate enough volatility in hours worked (the "labor market puzzle").

#### 4.2 The Labor Wedge
The model's optimality condition for labor is that the MRS between leisure and consumption must equal the MPL. We can use data to measure the implied **labor wedge**, the gap between these two quantities: $\text{Labor Wedge}_t = MPL_t / MRS_{L,C}$. If the model were perfect, this wedge would be 1. In reality, it is large and countercyclical, suggesting the model is missing important labor market frictions.

### 5. Chapter Summary
- **RBC Core Idea:** Business cycles are the efficient equilibrium response of the economy to real technology shocks.
- **First Welfare Theorem:** The competitive equilibrium of the baseline RBC model is Pareto optimal, identical to the solution of a Social Planner's problem.
- **Solution Method:** Models are typically solved by **log-linearizing** the equilibrium conditions around a non-stochastic steady state. This converts the non-linear system into a linear one that is easier to solve, often with algorithms like Uhlig's method.
- **News Shocks:** Modern RBC models emphasize the role of news about the future in generating business cycles. News shocks can generate investment-led booms that precede actual changes in productivity.
- **Critique:** While influential, the baseline RBC model struggles to match key features of the data, particularly the volatility of hours worked. The **labor wedge** points to missing labor market frictions, motivating New Keynesian models.

### 6. Exercises

1.  **Log-Linearization:** Manually log-linearize the firm's FOC for labor, $w_t = (1-\alpha) A_t K_t^\alpha L_t^{-\alpha}$, to get an equation in terms of log-deviations from the steady state.

2.  **Labor Supply Elasticity:** The parameter `phi` is the inverse of the Frisch elasticity of labor supply. Our baseline model used `phi=1.0`. Rerun the impulse response analysis for a surprise shock with a much higher `phi=5.0` (more inelastic labor supply). How does the impulse response of labor and output change? Explain the economic intuition.

3.  **Government Spending Shock:** Add a government that finances spending $G_t$ with lump-sum taxes $T_t$. The household budget constraint becomes $C_t + K_{t+1} = w_t L_t + (1+r_t-\delta) K_t - T_t$, and the resource constraint is $Y_t = C_t + I_t + G_t$. If there is a sudden, temporary increase in government spending ($G_t$), what would you predict happens to output, consumption, and investment? Explain the concept of "crowding out" in this context.

4. **News vs. Surprise:** In the comparison plot, why does consumption jump up immediately in response to a news shock, while it rises more slowly in response to a surprise shock? Relate this to the household's Euler equation.

5.  **Investment-Specific Shocks:** Introduce an investment-specific technology shock $Q_t$ into the capital accumulation equation: $K_{t+1} = (1-\delta)K_t + Q_t I_t$. A positive shock to $Q_t$ lowers the effective price of investment. How would you expect the economy to respond to such a shock compared to a neutral TFP shock?